In [53]:
import sys; sys.path.insert(0, '..')

from src.models.mnist_module import MNISTLitModule

In [115]:
from omegaconf import OmegaConf

dd = OmegaConf.load('../configs/experiment/example.yaml')

In [116]:
dd

{'defaults': [{'override /datamodule': 'mnist.yaml'}, {'override /model': 'mnist.yaml'}, {'override /callbacks': 'default.yaml'}, {'override /logger': None}, {'override /trainer': 'default.yaml'}], 'name': 'simple_dense_net', 'seed': 12345, 'trainer': {'min_epochs': 10, 'max_epochs': 10, 'gradient_clip_val': 0.5}, 'model': {'lr': 0.002, 'net': {'lin1_size': 128, 'lin2_size': 256, 'lin3_size': 64}}, 'datamodule': {'batch_size': 64}, 'logger': {'wandb': {'tags': ['mnist', '${name}']}}}

In [64]:
import ast

with open("../src/models/mnist_module.py", "r") as source:
    tree = ast.parse(source.read())

model_name = 'mnist'
# class_names = [f'{model_name}{m}' for m in ['model', 'module']]
classes = [n for n in tree.body if isinstance(n, ast.ClassDef)]
imports = [n for n in tree.body if isinstance(n, ast.Import)]
imports_from = [n for n in tree.body if isinstance(n, ast.ImportFrom)]
print([name.name for imp in imports for name in imp.names])
print([f'{imp.module}: {name.name}' for imp in imports_from for name in imp.names])
for class_ in classes:
    print(class_.name)
    if model_name in class_.name.lower():
        init = next((n for n in class_.body if isinstance(n, ast.FunctionDef) and n.name == '__init__'))
        args = init.args.args
        defaults = [None]*(len(args)-len(init.args.defaults)) + [d.value for d in init.args.defaults]
        annotations = [arg.annotation.id if isinstance(arg.annotation, ast.Name) else None for arg in args]
        print(annotations)

        for arg, df in zip(args, defaults):
            print(f'arg: {arg.arg}  default: {df}')

        break



['torch']
['typing: Any', 'typing: List', 'pytorch_lightning: LightningModule', 'torchmetrics: MaxMetric', 'torchmetrics.classification.accuracy: Accuracy', 'src.models.components.simple_dense_net: SimpleDenseNet']
MNISTLitModule
[None, 'SimpleDenseNet', 'float', 'float']
arg: self  default: None
arg: net  default: None
arg: lr  default: 0.001
arg: weight_decay  default: 0.0005


In [65]:
annotations

[None, 'SimpleDenseNet', 'float', 'float']

In [106]:
import ast


def recursively_update_class_arguments(argument_dict, class_name, module, prefix='../', add_target=True):
    
    with open(prefix + module.replace('.', '/') + '.py', "r") as source:
        tree = ast.parse(source.read())
    
    imports_from = [n for n in tree.body if isinstance(n, ast.ImportFrom)]
    import_dict = {name.name: imp.module for imp in imports_from for name in imp.names}
    classes = [n for n in tree.body if isinstance(n, ast.ClassDef)]
    for class_ in classes:
        if class_.name == class_name:
            if add_target:
                argument_dict['_target_'] = f'{module}.{class_name}'
            init = next((n for n in class_.body if isinstance(n, ast.FunctionDef) and n.name == '__init__'))
            if init is None:
                return
            args = init.args.args[1:]  # drop self
            defaults = ['???']*(len(args)-len(init.args.defaults)) + [ast.literal_eval(df) for df in init.args.defaults]
            annotations = [arg.annotation.id if isinstance(arg.annotation, ast.Name) else None for arg in args]
            
            for arg, dft, ann in zip(args, defaults, annotations):
                if dft == '???' and ann is not None and import_dict.get(ann) is not None:
                    argument_dict[arg.arg] = {}
                    recursively_update_class_arguments(argument_dict[arg.arg], ann, import_dict[ann])
                else:
                    argument_dict[arg.arg] = dft


In [107]:
df

0.0005

In [111]:
d = {}
recursively_update_class_arguments(d, 'MNISTLitModule', 'src.models.mnist_module')
dd = {}
recursively_update_class_arguments(dd, 'MNISTDataModule', 'src.datamodules.mnist_datamodule')


In [112]:
dd

{'_target_': 'src.datamodules.mnist_datamodule.MNISTDataModule',
 'data_dir': 'data/',
 'train_val_test_split': (55000, 5000, 10000),
 'batch_size': 64,
 'num_workers': 0,
 'pin_memory': False}

In [89]:
import yaml

with open('pippo.yaml', 'w') as p:
    yaml.dump(d, p)

In [78]:
ugo = list(d.keys())[0]
ugo.arg

'self'

In [31]:
def show_info(functionNode):
    print("Function name:", functionNode.name)
    print("Args:")
    for arg, def_val in zip(functionNode.args.args, functionNode.args.defaults):
        #import pdb; pdb.set_trace()
        print(f"\tParameter name: {arg.arg} \t default value: {def_val.value}")


# filename = "untrusted.py"
with open("../src/models/mnist_module.py", "r") as file:
    node = ast.parse(file.read())

functions = [n for n in node.body if isinstance(n, ast.FunctionDef)]
classes = [n for n in node.body if isinstance(n, ast.ClassDef)]

for function in functions:
    show_info(function)

for class_ in classes:
    print("Class name:", class_.name)
    methods = [n for n in class_.body if isinstance(n, ast.FunctionDef)]
    for method in methods:
        show_info(method)

Class name: MNISTLitModule
Function name: __init__
Args:
	Parameter name: self 	 default value: 0.001
	Parameter name: net 	 default value: 0.0005
Function name: forward
Args:
Function name: step
Args:
Function name: training_step
Args:
Function name: training_epoch_end
Args:
Function name: validation_step
Args:
Function name: validation_epoch_end
Args:
Function name: test_step
Args:
Function name: test_epoch_end
Args:
Function name: on_epoch_end
Args:
Function name: configure_optimizers
Args:


In [41]:
method = methods[0]

In [45]:
print(method.args.args[1].annotation)

In [25]:
method.name

'__init__'